<a href="https://colab.research.google.com/github/GeorgeM2000/CANE/blob/master/code/CANE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Libraries & Tools***

In [42]:
import argparse
import os
import random
import numpy as np
import tensorflow as tf
import zipfile
import gc
import shutil

from math import pow
from datetime import datetime
from pathlib import Path
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

In [ ]:
!pwd

/content


# ***Global Variables & General Functionality***

In [27]:
MAX_LEN = 300 # Default value for single execution
MAX_LENS = [] # List to hold the values for multiple execution
neg_table_size = 1000000
NEG_SAMPLE_POWER = 0.75
batch_size = 64
num_epoch = 50 # Default: 200
embed_size = 200
lr = 1e-3

In [28]:
dataset_name = "cora"
data_text_file = "YAKE.txt"
data_text_files = ["YAKE10.txt", "PositionRank.txt", "PositionRank10.txt"]
graph_file = 'graph.txt'
parent_path = f'/content/Datasets/{dataset_name}'
log_file = 'CANE_Execution_Logs.txt'
link_pred_results_file = 'CANE_Link_Pred_Res.txt'
node_clf_results_file = 'CANE_Node_Clf_Res.txt'
categories_file = 'group-v3.txt'


split_graph_file = 'sgraph15.txt'
split_graph_files = ['sgraph15.txt', 'sgraph45.txt', 'sgraph75.txt']
test_graph_file = 'tgraph85.txt'
test_graph_files = ['tgraph85.txt', 'tgraph55.txt', 'tgraph25.txt']

# Model parameters
ratio = 0.15
rho = "1.0,0.3,0.3"

In [29]:
clf_ratio = [0.15, 0.45, 0.75]
clf_num = 5
train_classifier = True

In [ ]:
for tf in data_text_files:
  max_word_count = 0
  min_word_count = float('inf')

  with open(f'{parent_path}/{tf}', 'r') as file:
      for line in file:
          word_count = len(line.split())

          if word_count > max_word_count:
              max_word_count = word_count

          if word_count < min_word_count:
              min_word_count = word_count

  MAX_LENS.append(max_word_count+1)
  print(f'=== {tf} ===')
  print("Max word count:", max_word_count)
  print("Min word count:", min_word_count)
  print()

gc.collect()

In [ ]:
MAX_LENS

In [ ]:
MAX_LEN = MAX_LENS[-1] # For single execution

In [ ]:
# Open and extract the zip file
with zipfile.ZipFile('/content/PartialData.zip', 'r') as zip_ref:
    zip_ref.extractall(parent_path)

print("Extraction complete!")

Extraction complete!


In [31]:
zero_list = []
for i in range(0, embed_size):
    zero_list.append(0)
zero_list = np.array(zero_list)

In [32]:
def get_vectors_from_file(file_path):
  vectors = {}

  with open(f'{file_path}', "r") as f:
      for idx, line in enumerate(f):
          vector = list(map(float, line.strip().split()))  # Convert to list of floats
          vectors[idx] = vector  # Assign embedding to node idx

  return vectors

Use the python code below only for node classification tasks

In [33]:
# Create the edge list. Store the unique nodes in the list "nodes"
with open(f'{parent_path}/{graph_file}', 'r') as f:
  eedges = f.readlines()

edge_list = []
nodes = [] # "nodes" will contain all the unique nodes of the graph
for ee in eedges:
  edge_list.append(list(ee.split()))
for ll in edge_list:
  for ed in ll:
    if ed not in nodes:
      nodes.append(ed)
    else:
      continue

In [ ]:
len(nodes)

In [ ]:
len(edge_list)

# ***DataSet***

In [41]:
class dataSet:
    def __init__(self, text_path, graph_path):

        text_file, graph_file = self.load(text_path, graph_path)
        self.edges = self.load_edges(graph_file)
        self.text, self.num_vocab, self.num_nodes = self.load_text(text_file)
        self.negative_table = InitNegTable(self.edges)

    def load(self, text_path, graph_path):
        text_file = open(text_path, 'rb').readlines()
        for a in range(0, len(text_file)):
            text_file[a] = str(text_file[a])
        graph_file = open(graph_path, 'rb').readlines()

        return text_file, graph_file

    def load_edges(self, graph_file):
        edges = []
        for i in graph_file:
            edges.append(list(map(int, i.strip().decode().split('\t'))))

        #print("Total load %d edges." % len(edges))

        return edges

    def load_text(self, text_file):
        """
        Adapting with adapt(text_data):

        vectorize_layer.adapt(text_data) analyzes text_data, builds a vocabulary, and assigns a unique integer ID to each word based on its frequency (most frequent words get lower IDs).
        Transforming with vectorize_layer(text_data):

        This maps each word in text_data to its corresponding integer token ID, producing a 2D array where each row represents a sequence of token IDs for a given input line, padded or truncated to max_len.
        """
        vectorize_layer = tf.keras.layers.TextVectorization(
            max_tokens=None,  # Set a limit if needed
            output_mode='int',
            output_sequence_length=MAX_LEN
        )

        text_data = [line.strip() for line in text_file]

        vectorize_layer.adapt(text_data)

        text = vectorize_layer(text_data).numpy()

        num_vocab = len(vectorize_layer.get_vocabulary())
        #print(f'Vocabulary: {num_vocab}')
        num_nodes = len(text)

        return text, num_vocab, num_nodes

    def negative_sample(self, edges):
        node1, node2 = zip(*edges)
        sample_edges = []
        func = lambda: self.negative_table[random.randint(0, neg_table_size - 1)]
        for i in range(len(edges)):
            neg_node = func()
            while node1[i] == neg_node or node2[i] == neg_node:
                neg_node = func()
            sample_edges.append([node1[i], node2[i], neg_node])

        return sample_edges

    def generate_batches(self, mode=None):

        num_batch = len(self.edges) // batch_size
        edges = self.edges

        if mode == 'add':
          num_batch += 1
          edges.extend(edges[:(batch_size - len(self.edges) // batch_size)])

        if mode != 'add':
          random.shuffle(edges)
        sample_edges = edges[:num_batch * batch_size]
        sample_edges = self.negative_sample(sample_edges)

        batches = []
        for i in range(num_batch):
            batches.append(sample_edges[i * batch_size:(i + 1) * batch_size])
        # print sample_edges[0]
        return batches


# ***CANE***

In [8]:
class Model:
    def __init__(self, vocab_size, num_nodes, rho):
        rho = rho.split(",")
        self.rho1 = float(rho[0])
        self.rho2 = float(rho[1])
        self.rho3 = float(rho[2])

        # '''hyperparameter'''
        with tf.name_scope('read_inputs') as scope:
          self.Text_a = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Ta')
          self.Text_b = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Tb')
          self.Text_neg = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Tneg')
          self.Node_a = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n1')
          self.Node_b = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n2')
          self.Node_neg = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n3')

        with tf.name_scope('initialize_embedding') as scope:
            self.text_embed = tf.Variable(tf.random.truncated_normal([vocab_size, embed_size // 2], stddev=0.3))
            self.node_embed = tf.clip_by_norm(tf.Variable(tf.random.truncated_normal([num_nodes, embed_size // 2], stddev=0.3)), clip_norm=1, axes=1)

        with tf.name_scope('lookup_embeddings') as scope:
            self.TA = tf.nn.embedding_lookup(self.text_embed, self.Text_a)
            self.T_A = tf.expand_dims(self.TA, -1)

            self.TB = tf.nn.embedding_lookup(self.text_embed, self.Text_b)
            self.T_B = tf.expand_dims(self.TB, -1)

            self.TNEG = tf.nn.embedding_lookup(self.text_embed, self.Text_neg)
            self.T_NEG = tf.expand_dims(self.TNEG, -1)

            self.N_A = tf.nn.embedding_lookup(self.node_embed, self.Node_a)
            self.N_B = tf.nn.embedding_lookup(self.node_embed, self.Node_b)
            self.N_NEG = tf.nn.embedding_lookup(self.node_embed, self.Node_neg)

        self.convA, self.convB, self.convNeg = self.conv()
        self.loss = self.compute_loss()

    def conv(self):
        W2 = tf.Variable(tf.random.truncated_normal([2, embed_size // 2, 1, 100], stddev=0.3))
        rand_matrix = tf.Variable(tf.random.truncated_normal([100, 100], stddev=0.3))

        convA = tf.nn.conv2d(self.T_A, W2, strides=[1, 1, 1, 1], padding='VALID')
        convB = tf.nn.conv2d(self.T_B, W2, strides=[1, 1, 1, 1], padding='VALID')
        convNEG = tf.nn.conv2d(self.T_NEG, W2, strides=[1, 1, 1, 1], padding='VALID')

        hA = tf.tanh(tf.squeeze(convA))
        hB = tf.tanh(tf.squeeze(convB))
        hNEG = tf.tanh(tf.squeeze(convNEG))

        tmphA = tf.reshape(hA, [batch_size * (MAX_LEN - 1), embed_size // 2])
        ha_mul_rand = tf.reshape(tf.matmul(tmphA, rand_matrix),
                                 [batch_size, MAX_LEN - 1, embed_size // 2])
        r1 = tf.matmul(ha_mul_rand, hB, adjoint_b=True)
        r3 = tf.matmul(ha_mul_rand, hNEG, adjoint_b=True)
        att1 = tf.expand_dims(tf.stack(r1), -1)
        att3 = tf.expand_dims(tf.stack(r3), -1)

        att1 = tf.tanh(att1)
        att3 = tf.tanh(att3)

        pooled_A = tf.reduce_mean(att1, 2)
        pooled_B = tf.reduce_mean(att1, 1)
        pooled_NEG = tf.reduce_mean(att3, 1)

        a_flat = tf.squeeze(pooled_A)
        b_flat = tf.squeeze(pooled_B)
        neg_flat = tf.squeeze(pooled_NEG)

        w_A = tf.nn.softmax(a_flat)
        w_B = tf.nn.softmax(b_flat)
        w_NEG = tf.nn.softmax(neg_flat)

        rep_A = tf.expand_dims(w_A, -1)
        rep_B = tf.expand_dims(w_B, -1)
        rep_NEG = tf.expand_dims(w_NEG, -1)

        hA = tf.transpose(hA, perm=[0, 2, 1])
        hB = tf.transpose(hB, perm=[0, 2, 1])
        hNEG = tf.transpose(hNEG, perm=[0, 2, 1])

        rep1 = tf.matmul(hA, rep_A)
        rep2 = tf.matmul(hB, rep_B)
        rep3 = tf.matmul(hNEG, rep_NEG)

        attA = tf.squeeze(rep1)
        attB = tf.squeeze(rep2)
        attNEG = tf.squeeze(rep3)

        return attA, attB, attNEG

    def compute_loss(self):
        p1 = tf.reduce_sum(tf.multiply(self.convA, self.convB), 1)
        p1 = tf.math.log(tf.nn.sigmoid(p1) + 0.001)

        p2 = tf.reduce_sum(tf.multiply(self.convA, self.convNeg), 1)
        p2 = tf.math.log(tf.nn.sigmoid(-p2) + 0.001)

        p3 = tf.reduce_sum(tf.multiply(self.N_A, self.N_B), 1)
        p3 = tf.math.log(tf.nn.sigmoid(p3) + 0.001)

        p4 = tf.reduce_sum(tf.multiply(self.N_A, self.N_NEG), 1)
        p4 = tf.math.log(tf.nn.sigmoid(-p4) + 0.001)

        p5 = tf.reduce_sum(tf.multiply(self.convB, self.N_A), 1)
        p5 = tf.math.log(tf.nn.sigmoid(p5) + 0.001)

        p6 = tf.reduce_sum(tf.multiply(self.convNeg, self.N_A), 1)
        p6 = tf.math.log(tf.nn.sigmoid(-p6) + 0.001)

        p7 = tf.reduce_sum(tf.multiply(self.N_B, self.convA), 1)
        p7 = tf.math.log(tf.nn.sigmoid(p7) + 0.001)

        p8 = tf.reduce_sum(tf.multiply(self.N_B, self.convNeg), 1)
        p8 = tf.math.log(tf.nn.sigmoid(-p8) + 0.001)

        rho1 = self.rho1
        rho2 = self.rho2
        rho3 = self.rho3
        temp_loss = rho1 * (p1 + p2) + rho2 * (p3 + p4) + rho3 * (p5 + p6) + rho3 * (p7 + p8)
        loss = -tf.reduce_sum(temp_loss)
        return loss

# ***Negative Sample***

In [9]:
def InitNegTable(edges):
    a_list, b_list = zip(*edges)
    a_list = list(a_list)
    b_list = list(b_list)
    node = a_list
    node.extend(b_list)

    node_degree = {}
    for i in node:
        if i in node_degree:
            node_degree[i] += 1
        else:
            node_degree[i] = 1
    sum_degree = 0
    for i in node_degree.values():
        sum_degree += pow(i, 0.75)

    por = 0
    cur_sum = 0
    vid = -1
    neg_table = []
    degree_list = list(node_degree.values())
    node_id = list(node_degree.keys())
    for i in range(neg_table_size):
        if ((i + 1) / float(neg_table_size)) > por:
            cur_sum += pow(degree_list[vid + 1], NEG_SAMPLE_POWER)
            por = cur_sum / sum_degree
            vid += 1
        neg_table.append(node_id[vid])

    #print(len(neg_table))
    return neg_table


# ***Classify***

In [10]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return np.asarray(all_labels)


class Classifier(object):

    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        # X_train = [self.embeddings[x] for x in X]
        X_train = [self.embeddings[int(x)] for x in X] # For each node in X, take its embedding
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y] # For each label in Y, take its size (multi-label)
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        return results

    def predict(self, X, top_k_list):
        X_ = np.asarray([self.embeddings[int(x)] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = np.random.get_state()
        training_size = int(train_precent * len(X)) # Set the ratio based on the size of X
        np.random.seed(seed)
        shuffle_indices = np.random.permutation(np.arange(len(X))) # Shuffle the indices of X (X contains all nodes)

        # Access the values of X and Y based on the shuffled indices

        # X_train and Y_train will have "training_size" number of values of X and Y
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]

        # X_test and Y_test will have "len(X) - training_size" number of values of X and Y
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y) # Y has the labels of all nodes
        np.random.set_state(state)
        return self.evaluate(X_test, Y_test)



def load_embeddings(filename):
    fin = open(filename, 'r')
    node_num, size = [int(x) for x in fin.readline().strip().split()]
    vectors = {}
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split(' ')
        assert len(vec) == size + 1
        vectors[vec[0]] = [float(x) for x in vec[1:]]
    fin.close()
    assert len(vectors) == node_num
    return vectors

def read_node_label(filename):
    fin = open(filename, 'r')
    X = []
    Y = []
    XY_dic = {}
    X_Y_dic = {}
    while 1:
        l = fin.readline()
        if l == '':
            break
        # vec = l.strip().split('\t')
        vec = l.strip().split(' ')
        X.append(vec[0])
        Y.append(vec[1:])
        X_Y_dic[str(vec[0])] = str(vec[1:][0])
        XY_dic.setdefault(str(vec[1:][0]), []).append(str(vec[0]))
    fin.close()
    return X, Y, XY_dic, X_Y_dic

# ***Run (Single execution)***

In [ ]:
def prepareData(graph, ratio):
  with open(f'{parent_path}/{graph}', 'rb') as f:
    edges = [i for i in f]

  selected = int(len(edges) * float(ratio))
  selected = selected - selected % batch_size
  selected = random.sample(edges, selected)
  remain = [i for i in edges if i not in selected]
  try:
    temp_dir = Path('temp')

    # Check if the directory exists, if so, delete it
    if temp_dir.exists() and temp_dir.is_dir():
        shutil.rmtree(temp_dir)
        print("Existing directory deleted.")

    # Create the directory
    temp_dir.mkdir(parents=True, exist_ok=True)
    print("Directory created successfully.")

  except Exception as e:
      print(f"An error occurred: {e}")

  with open('temp/graph.txt', 'wb') as f:
    for i in selected:
      f.write(i)

  with open('temp/test_graph.txt', 'wb') as f:
    for i in remain:
      f.write(i)

In [ ]:
prepareData(graph_file, ratio)

In [23]:
# Load data
#graph_path = os.path.join('/content/temp/graph.txt') # Use this if you executed the prepareData() function

data = dataSet(f'{parent_path}/{data_text_file}',
               f'{parent_path}/{graph_file}')

# Saving embeddings
#embed_file = f"{parent_path}/Results/CANE/embed_link_pred_{split_graph_file.split('.')[0]}_{data_text_file.split('.')[0]}.txt"
embed_file = f"{parent_path}/Results/CANE/embed_node_clf_{graph_file.split('.')[0]}_{data_text_file.split('.')[0]}.txt" # For node classification the whole graph ('graph.txt') is used

In [ ]:
with tf.Graph().as_default():
    sess = tf.compat.v1.Session()
    with sess.as_default():
        model = Model(data.num_vocab, data.num_nodes, rho)
        opt = tf.compat.v1.train.AdamOptimizer(lr)
        train_op = opt.minimize(model.loss)
        sess.run(tf.compat.v1.global_variables_initializer())
        #total_time = 0

        # Training
        print('Start training.......')
        start_time = datetime.now()
        for epoch in tqdm(range(num_epoch)):
            #start_time = datetime.now()
            loss_epoch = 0
            batches = data.generate_batches()
            h1 = 0
            num_batch = len(batches)
            for i in range(num_batch):
                batch = batches[i]

                node1, node2, node3 = zip(*batch)
                node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                feed_dict = {
                    model.Text_a: text1,
                    model.Text_b: text2,
                    model.Text_neg: text3,
                    model.Node_a: node1,
                    model.Node_b: node2,
                    model.Node_neg: node3
                }

                # run the graph
                _, loss_batch = sess.run([train_op, model.loss], feed_dict=feed_dict)
                loss_epoch += loss_batch

            #end_time = datetime.now()
            #total_time += (end_time - start_time).total_seconds()
            #print('epoch: ', epoch + 1, ' loss: ', loss_epoch)

        end_time = datetime.now()
        print(f'Total time: {((end_time - start_time).total_seconds()) / 60.0} min')

        with open(embed_file, 'wb') as f: # In a single execution setting only one "embeddings.txt" file is required
            batches = data.generate_batches(mode='add')
            num_batch = len(batches)
            embed = [[] for _ in range(data.num_nodes)]

            for i in range(num_batch):
                batch = batches[i]
                node1, node2, node3 = zip(*batch)
                node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                feed_dict = {
                    model.Text_a: text1,
                    model.Text_b: text2,
                    model.Text_neg: text3,
                    model.Node_a: node1,
                    model.Node_b: node2,
                    model.Node_neg: node3
                }

                # Fetch embeddings
                convA, convB, TA, TB = sess.run([model.convA, model.convB, model.N_A, model.N_B], feed_dict=feed_dict)

                for j in range(batch_size):
                    em = list(convA[j]) + list(TA[j])
                    #em = list(TA[j])
                    embed[node1[j]].append(em)
                    em = list(convB[j]) + list(TB[j])
                    #em = list(TB[j])
                    embed[node2[j]].append(em)


            for i in range(data.num_nodes):
                if embed[i]:
                    tmp = np.sum(embed[i], axis=0) / len(embed[i]) # np.mean(embed[i], axis=0)
                    f.write((' '.join(map(str, tmp)) + '\n').encode())
                else:
                    #f.write('\n'.encode())
                    f.write((' '.join(map(str, zero_list)) + '\n').encode()) # For node classification

gc.collect()

## ***Link Prediction***

In [ ]:
node2vec = {}
with open(embed_file, 'rb') as f:
  for i, j in enumerate(f):
    if j.decode() != '\n':
      node2vec[i] = list(map(float, j.strip().decode().split(' ')))


with open(os.path.join(f'{parent_path}/{test_graph_file}'), 'rb') as f:
  edges = [list(map(int, i.strip().decode().split('\t'))) for i in f]


nodes = list(set([i for j in edges for i in j]))
a = 0
b = 0
for i, j in edges:
  if i in node2vec.keys() and j in node2vec.keys():
    dot1 = np.dot(node2vec[i], node2vec[j])
    random_node = random.sample(nodes, 1)[0]
    while random_node == j or random_node not in node2vec.keys():
        random_node = random.sample(nodes, 1)[0]
    dot2 = np.dot(node2vec[i], node2vec[random_node])
    if dot1 > dot2:
        a += 1
    elif dot1 == dot2:
        a += 0.5
    b += 1

print("Auc value:", float(a) / b)

Auc value: 0.9501915708812261


## ***Node Classification***

In [25]:
with open(f'{parent_path}/{categories_file}', 'r') as f:
  tags = f.readlines() # "tags" will be a 2D list. Each sublist will have the form: nodeID     label

if train_classifier:

  clf_test_len = len(nodes) # The number of nodes will be the same in each run since we're using the whole graph and thus, all of its nodes
  print('Train classifier start!')

  X = []
  Y = []
  new_vector = get_vectors_from_file(embed_file)

  for jk in range(0, clf_test_len):
    if str(jk) in nodes: # If the index "jk" is a node
      tag_list = tags[jk].strip().split() # For node "jk", take this info: jk     label
      # Y.append([(int)(i) for i in tags])
      lli = [str(i) for i in tag_list] # For node "jk", lli will contain all of its labels
      if len(lli) != 0:
        if np.array(new_vector[jk]).any() != np.array(zero_list).any(): # If there is no zero value in the embedding of "jk"
          X.append(jk)
          Y.append(lli[1:][0]) # Take the first label (if there are multiple) of node "jk"

  # This part of the code uses only the X and Y lists created above
  mi = {}
  ma = {}
  li1 = []
  li2 = []
  with open(f'{parent_path}/Results/CANE/{node_clf_results_file}', 'a') as f:
    f.write(f"{embed_file.split('/')[-1]} \n")
    print(embed_file.split('/')[-1])
    for i in range(0, len(clf_ratio)): # Experiment with each ratio
      for j in range(0, clf_num): # clf_num = 5

        clf = Classifier(vectors=new_vector, # All node embeddings
                        clf=LogisticRegression())

        result = clf.split_train_evaluate(X, Y, clf_ratio[i])

        # Results
        li1.append(result['micro'])
        li2.append(result['macro'])


      mi[str(str(clf_ratio[i]) + '-micro')] = sum(li1) / clf_num
      ma[str(str(clf_ratio[i]) + '-macro')] = sum(li2) / clf_num

      print(mi)
      print(ma)
      print()

      f.writelines(str(str(mi)+str(ma)))
      f.write('\n')

      # Reinitialize the dictionaries and lists
      mi = {}
      ma = {}
      li1 = []
      li2 = []



Train classifier start!
embed_node_clf_graph_YAKE.txt
{'0.15-micro': 0.7486308871851041}
{'0.15-macro': 0.7434241299869686}

{'0.45-micro': 0.7698815566835872}
{'0.45-macro': 0.7699308483872019}

{'0.75-micro': 0.7839851024208566}
{'0.75-macro': 0.7879445450861173}



# ***Run (Multiple executions)***

In [ ]:
for gf in ['graph.txt']: #for gf in split_graph_files: # For link prediction. For node classification just use: for gf in ['graph.txt']:
    for t, txtf in enumerate(data_text_files):

        MAX_LEN = MAX_LENS[t]
        print(f'The maximum length is: {MAX_LEN}')

        data = dataSet(f'{parent_path}/{txtf}', f'{parent_path}/{gf}')

        # Logging the execution details
        with open(f'{parent_path}/Results/CANE/{log_file}', 'a') as f:
            f.write(f'Processing graph: {gf}, text: {txtf}\n')

        print(f'Processing graph: {gf}, text: {txtf}')

        with tf.Graph().as_default():
            sess = tf.compat.v1.Session()
            with sess.as_default():
                model = Model(data.num_vocab, data.num_nodes, rho)
                opt = tf.compat.v1.train.AdamOptimizer(lr)
                train_op = opt.minimize(model.loss)
                sess.run(tf.compat.v1.global_variables_initializer())
                #total_time = 0

                # Training
                start_time = datetime.now()
                for epoch in tqdm(range(num_epoch)):
                    #start_time = datetime.now()
                    loss_epoch = 0
                    batches = data.generate_batches()
                    num_batch = len(batches)

                    for i in range(num_batch):
                        batch = batches[i]
                        node1, node2, node3 = zip(*batch)
                        node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                        text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                        feed_dict = {
                            model.Text_a: text1,
                            model.Text_b: text2,
                            model.Text_neg: text3,
                            model.Node_a: node1,
                            model.Node_b: node2,
                            model.Node_neg: node3
                        }

                        # Run the graph
                        _, loss_batch = sess.run([train_op, model.loss], feed_dict=feed_dict)
                        loss_epoch += loss_batch

                    #end_time = datetime.now()
                    #epoch_time = (end_time - start_time).total_seconds()
                    #total_time += epoch_time

                end_time = datetime.now()
                with open(f'{parent_path}/Results/CANE/{log_file}', 'a') as f:
                    f.write(f'Time: {((end_time - start_time).total_seconds()) / 60.0}\n')

                print(f'Total Time: {((end_time - start_time).total_seconds()) / 60.0} min')

                # Save embeddings with a unique name
                #embed_file = f"{parent_path}/Results/CANE/embed_link_pred_{gf.split('.')[0]}_{txtf.split('.')[0]}.txt"
                embed_file = f"{parent_path}/Results/CANE/embed_node_clf_{gf.split('.')[0]}_{txtf.split('.')[0]}.txt"

                with open(embed_file, 'wb') as f:
                    batches = data.generate_batches(mode='add')
                    num_batch = len(batches)
                    embed = [[] for _ in range(data.num_nodes)]

                    for i in range(num_batch):
                        batch = batches[i]
                        node1, node2, node3 = zip(*batch)
                        node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                        text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                        feed_dict = {
                            model.Text_a: text1,
                            model.Text_b: text2,
                            model.Text_neg: text3,
                            model.Node_a: node1,
                            model.Node_b: node2,
                            model.Node_neg: node3
                        }

                        # Fetch embeddings
                        convA, convB, TA, TB = sess.run(
                            [model.convA, model.convB, model.N_A, model.N_B],
                            feed_dict=feed_dict
                        )

                        for j in range(batch_size):
                            em = list(TA[j])
                            embed[node1[j]].append(em)
                            em = list(TB[j])
                            embed[node2[j]].append(em)

                    for i in range(data.num_nodes):
                        if embed[i]:
                            tmp = np.sum(embed[i], axis=0) / len(embed[i]) # np.mean(embed[i], axis=0)
                            f.write((' '.join(map(str, tmp)) + '\n').encode())
                        else:
                            f.write('\n'.encode()) # For link prediction
                            #f.write((' '.join(map(str, zero_list)) + '\n').encode()) # For node classification

                # Log completion
                with open(f'{parent_path}/Results/CANE/{log_file}', 'a') as f:
                    f.write(f'Embeddings saved to: {embed_file}\n')

        gc.collect()

## ***Node Classification***

In [ ]:
embed_files = [f'{parent_path}/Results/CANE/embed_node_clf_graph_YAKE10.txt',
               f'{parent_path}/Results/CANE/embed_node_clf_graph_PositionRank.txt',
               f'{parent_path}/Results/CANE/embed_node_clf_graph_PositionRank10.txt']

with open(f'{parent_path}/{categories_file}', 'r') as f:
  tags = f.readlines() # "tags" will be a 2D list. Each sublist will have the form: nodeID     label

if train_classifier:

  clf_test_len = len(nodes) # The number of nodes will be the same in each run since we're using the whole graph and thus, all of its nodes
  print('Train classifier start!')

  for ef in embed_files:
    X = []
    Y = []
    new_vector = get_vectors_from_file(ef)

    for jk in range(0, clf_test_len):
      if str(jk) in nodes: # If the index "jk" is a node
        tag_list = tags[jk].strip().split() # For node "jk", take this info: jk     label
        # Y.append([(int)(i) for i in tags])
        lli = [str(i) for i in tag_list] # For node "jk", lli will contain all of its labels
        if len(lli) != 0:
          if np.array(new_vector[jk]).any() != np.array(zero_list).any(): # If there is no zero value in the embedding of "jk"
            X.append(jk)
            Y.append(lli[1:][0]) # Take the first label (if there are multiple) of node "jk"

    # This part of the code uses only the X and Y lists created above
    mi = {}
    ma = {}
    li1 = []
    li2 = []
    with open(f'{parent_path}/Results/CANE/{node_clf_results_file}', 'a') as f:
      f.write(f"{ef.split('/')[-1]} \n")
      print(ef.split('/')[-1])
      for i in range(0, len(clf_ratio)): # Experiment with each ratio
        for j in range(0, clf_num): # clf_num = 5

          clf = Classifier(vectors=new_vector, # All node embeddings
                          clf=LogisticRegression())

          result = clf.split_train_evaluate(X, Y, clf_ratio[i])

          # Results
          li1.append(result['micro'])
          li2.append(result['macro'])


        mi[str(str(clf_ratio[i]) + '-micro')] = sum(li1) / clf_num
        ma[str(str(clf_ratio[i]) + '-macro')] = sum(li2) / clf_num

        print(mi)
        print(ma)
        print()

        f.writelines(str(str(mi)+str(ma)))
        f.write('\n')

        # Reinitialize the dictionaries and lists
        mi = {}
        ma = {}
        li1 = []
        li2 = []

    gc.collect()

## ***Link Prediction***

In [ ]:
embed_files = [[f'{parent_path}/Results/CANE/embed_link_pred_sgraph15_RAKE5.txt']]

# Initialize a log file to store the AUC results
with open(f'{parent_path}/Results/CANE/{link_pred_results_file}', "a") as f:
    f.write("Embed File\tAUC Value\n")

for tgfi, tgf in enumerate(test_graph_files):
  for ef in embed_files[tgfi]:
      node2vec = {}

      # Load the embeddings from the current embed file
      with open(ef, 'rb') as f:
          for i, j in enumerate(f):
              if j.decode().strip():
                  node2vec[i] = list(map(float, j.strip().decode().split(' ')))

      # Load the edges from the test graph file
      with open(f'{parent_path}/{tgf}', 'rb') as f:
          edges = [list(map(int, i.strip().decode().split())) for i in f]

      nodes = list(set([i for j in edges for i in j]))

      # Calculate AUC
      a = 0
      b = 0
      for i, j in edges:
          if i in node2vec.keys() and j in node2vec.keys():
              dot1 = np.dot(node2vec[i], node2vec[j])
              random_node = random.sample(nodes, 1)[0]
              while random_node == j or random_node not in node2vec.keys():
                  random_node = random.sample(nodes, 1)[0]
              dot2 = np.dot(node2vec[i], node2vec[random_node])
              if dot1 > dot2:
                  a += 1
              elif dot1 == dot2:
                  a += 0.5
              b += 1

      auc_value = float(a) / b if b > 0 else 0
      print(f"AUC value for {ef.split('/')[-1]}: {auc_value}")

      # Log the result
      with open(f'{parent_path}/Results/CANE/{link_pred_results_file}', "a") as f:
          f.write(f"{ef}\t{tgf}\t{auc_value}\n")

      gc.collect()

AUC value for /content/datasets/arxiv/embed_sgraph15.txt_RAKE5.txt.txt: 0.9268285931769428
AUC value for /content/datasets/arxiv/embed_sgraph15.txt_data.txt.txt: 0.929254030862916
